In [ ]:
import nest_asyncio
nest_asyncio.apply()
from pathlib import Path

import sys
sys.path.append("../")

from desci_sense.shared_functions.init import init_multi_chain_parser_config
from desci_sense.shared_functions.parsers.multi_chain_parser import MultiChainParser
from desci_sense.evaluation.utils import get_dataset
from desci_sense.shared_functions.dataloaders import (
    scrape_post,
    convert_text_to_ref_post,
)

In [ ]:
config = init_multi_chain_parser_config(llm_type="google/gemma-7b-it",
                                        post_process_type="combined")

In [ ]:
multi_chain_parser = MultiChainParser(config)

2024-05-01 09:42:12.429 | INFO     | desci_sense.shared_functions.parsers.multi_chain_parser:__init__:64 - Initializing MultiChainParser. PostProcessType=combined
2024-05-01 09:42:12.452 | INFO     | desci_sense.shared_functions.parsers.multi_chain_parser:__init__:71 - Initializing post parsers...
2024-05-01 09:42:12.453 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:24 - Initializing parser chain 'refs_tagger' 
2024-05-01 09:42:12.598 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:24 - Initializing parser chain 'topics' 
2024-05-01 09:42:12.609 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:24 - Initializing parser chain 'keywords' 


In [ ]:
x= """
Did you think this was a conspiracy theory ?

* CDC Releases 780,000 Vaccine Injury Reports From V-safe System

V-Safe Data: https://ept.ms/43XPhLZ

and see:

The Pfizer Files: https://ept.ms/3VUYqTR
"""

Python(54517) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [ ]:
result = multi_chain_parser.process_ref_post(convert_text_to_ref_post(x),active_list=['topics','keywords'])

Python(54519) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(54520) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
2024-05-02 10:05:34.233 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation_async:20 - target_url=https://icandecide.org/pfizer-documents/
2024-05-02 10:05:34.244 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation_async:20 - target_url=https://icandecide.org/v-safe-data/
Python(54521) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(54522) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(54523) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(54543) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(54544) MallocStackLogging: can't turn off malloc stack logging because it was 

In [ ]:
# example raw text posts
TEST_POST_TEXT_W_REF = """
Hee he
"""

TEST_POST_2_TEXT_W_REF = """
I really disagree with this paper!
https://arxiv.org/abs/2402.04607
"""

TEXT_POSTS = [TEST_POST_TEXT_W_REF, TEST_POST_2_TEXT_W_REF]

In [ ]:
# convert raw text to RefPost format for input to the parser
inputs = [convert_text_to_ref_post(p) for p in TEXT_POSTS]

In [ ]:
# or scrape posts by url
urls = [
        "https://mastodon.social/@psmaldino@qoto.org/111405098400404613",
        "https://mastodon.social/@UlrikeHahn@fediscience.org/111732713776994953",
        "https://mastodon.social/@ronent/111687038322549430",
    ]
posts = [scrape_post(url) for url in urls]

In [ ]:
# batch process
results = multi_chain_parser.batch_process_ref_posts(inputs,active_list=["keywords", "topics"],batch_size=10)

In [ ]:
# single process
result = multi_chain_parser.process_ref_post(posts[0])

NameError: name 'posts' is not defined

In [ ]:
results[0].filter_classification.value

'not_research'

In [ ]:
result.reference_tagger

NameError: name 'result' is not defined

In [ ]:
result.topics

['academia', 'news', 'technology', 'research', 'health']

Python(54552) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [ ]:
result.item_types

['blogPost', 'blogPost']

Python(54556) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [ ]:
results[0].debug.keys()


dict_keys(['reference_tagger', 'topics', 'keywords'])

In [ ]:
"Keywords: "+results[0].debug['keywords']['raw_text']+" Topics: "+results[0].debug['topics']['full_text']


"Keywords: **Reasoning Steps:**\n\nThe provided text does not contain any meaningful content, making keyword extraction impractical. Therefore, I am unable to generate the requested annotations. Topics: **Reasoning Steps:**\n\n- The post contains a humorous expression, suggesting the topic might be related to entertainment or leisure.\n- The brevity of the post suggests a concise or punchy message.\n\n\n**Candidate Topics:**\n\n- Humor\n- Entertainment & Leisure\n\n\n**Final Answer:**\n\n**Humor** and **Entertainment & Leisure** \n\n ##Allowed terms: ['technology', 'science', 'academia', 'research', 'design', 'climate', 'sustainability', 'software & hardware', 'philosophy', 'health', 'culture', 'economics', 'business', 'politics', 'news', 'finance', 'sports', 'entertainment & leisure', 'art', 'literature', 'travel', 'personal', 'humour', 'other']"

## Loading dataset to play

In [ ]:
import wandb
import pandas as pd

wandb.login()

api = wandb.Api()

#TODO move from testing
run = wandb.init(project="testing", job_type="evaluation")

# get artifact path

dataset_artifact_id = (
        'common-sense-makers/filter_evaluation/labeled_tweets_no_threads:v1'
    )

# set artifact as input artifact
dataset_artifact = run.use_artifact(dataset_artifact_id)

# initialize table path
# add the option to call table_path =  arguments.get('--dataset')

# download path to table
a_path = dataset_artifact.download()
print("The path is",a_path)

# get dataset file name

table_path = Path(f"{a_path}/labeled_data_table_no_threads.table.json")


# return the pd df from the table
#remember to remove the head TODO
df = get_dataset(table_path)

table_path = Path(f"{a_path}/handles_chart.table.json")

df_handles = get_dataset(table_path)



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shahar-r-oriel (common-sense-makers). Use `wandb login --relogin` to force relogin


wandb:   2 of 2 files downloaded.  


The path is /Users/shaharorielkagan/sensemakers/nlp/notebooks/artifacts/labeled_tweets_no_threads:v1


In [ ]:
from tqdm import tqdm

In [ ]:
import concurrent.futures
from tqdm import tqdm

def process_text(text):
    return convert_text_to_ref_post(text)

# Assuming df['Text'] is your DataFrame column
with concurrent.futures.ThreadPoolExecutor() as executor:
    inputs = list(tqdm(executor.map(process_text, df['Text']), total=len(df['Text'])))


100%|██████████| 467/467 [00:42<00:00, 11.01it/s]


In [ ]:
bad_posts = []
for i in range(20,len(df["Text"])):
    print(i)
    p = df['Text'][i]
    r = convert_text_to_ref_post(p)
    try:
        result=multi_chain_parser.process_ref_post(r,active_list=["topics",'keywords'])
    except Exception as e:
        print("Post with text: ",p)
        print('and url: ',df['postURL'][i])
        print("has raised an exception: ",e) 
        bad_posts.append((df['postURL'][i],e))
          


    

20


2024-04-30 20:46:08.416 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation:54 - fetching citoid data for: https://www.youtube.com/watch?feature=youtu.be&si=XC_4OAHNNQM3BMLw&v=wS8Z3qKo5YU
2024-04-30 20:46:10.493 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:173 - Processing post with parsers: ['topics']
2024-04-30 20:46:10.497 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:175 - Instantiating prompts...
2024-04-30 20:46:10.505 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:180 - Invoking parallel chain...


21


2024-04-30 20:46:14.496 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation:54 - fetching citoid data for: https://www.youtube.com/watch?feature=youtu.be&si=ADozAL8LPmUvybux&v=3taXF-cjLBc
2024-04-30 20:46:16.460 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:173 - Processing post with parsers: ['topics']
2024-04-30 20:46:16.465 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:175 - Instantiating prompts...
2024-04-30 20:46:16.468 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:180 - Invoking parallel chain...
2024-04-30 20:46:19.247 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:173 - Processing post with parsers: ['topics']
2024-04-30 20:46:19.249 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:175 - Instantiating prompts...
2024-04-30 20:46:19.250 | DEBUG    | desci_sen

22
23


...

465


2024-04-30 21:28:15.723 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation:54 - fetching citoid data for: https://twitter.com/HalSinger/status/1778553827672662473
2024-04-30 21:28:15.955 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:173 - Processing post with parsers: ['topics']
2024-04-30 21:28:15.956 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:175 - Instantiating prompts...
2024-04-30 21:28:15.958 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:180 - Invoking parallel chain...
Python(27347) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


466


2024-04-30 21:28:19.955 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation:54 - fetching citoid data for: https://twitter.com/gateklons/status/1778477958790807562
Python(27348) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
2024-04-30 21:28:20.186 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:173 - Processing post with parsers: ['topics']
2024-04-30 21:28:20.187 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:175 - Instantiating prompts...
2024-04-30 21:28:20.187 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:180 - Invoking parallel chain...
Python(27349) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Python(27350) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(27351) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(27352) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(27353) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(27354) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(27355) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(27356) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(27357) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(27377) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(27378) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(27379) Malloc

In [ ]:
print(bad_posts)

[]


Python(28593) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(28594) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(28595) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(28596) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(28597) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(28598) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(28599) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(28600) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(28601) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(28602) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(28603) Malloc